1. Import Libary

In [ ]:
from datetime import datetime, timedelta, date
import pandas as pd
import pyodbc 
from sqlalchemy import create_engine, types
import urllib # Import riêng thư viện urllib

2. Import Data Frame From SQL SERVER

In [ ]:
# ==========================================
# 1. CẤU HÌNH KẾT NỐI (Thay đổi nếu cần)
# ==========================================
server = 'DESKTOP-MV8RPS5'
database = 'Data_Topic'
table_name = 'final_project_cleaned'

connection_string = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"Trusted_Connection=yes;"
    f"TrustServerCertificate=yes;"
)

params = urllib.parse.quote_plus(connection_string)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# ==========================================
# 2. KÍCH HOẠT CHẾ ĐỘ SIÊU TỐC (FAST_EXECUTEMANY)
# ==========================================
@event.listens_for(engine, "connect")
def receive_connect(dbapi_connection, connection_record):
    cursor = dbapi_connection.cursor()
    cursor.fast_executemany = True
    cursor.close()

# ==========================================
# 3. ĐỊNH NGHĨA KIỂU DỮ LIỆU (CHỐNG LỖI TRUNCATION)
# ==========================================
# Tự động tìm các cột có dữ liệu dài để đặt là NVARCHAR(MAX)
dtype_dict = {}
long_text_cols = ['devtype', 'languageworkedwith', 'platformworkedwith', 'ethnicity', 'devenviron']

for col in df_final_project.columns:
    if col.lower() in long_text_cols:
        dtype_dict[col] = types.NVARCHAR(length=None) # Tương đương MAX

# ==========================================
# 4. THỰC THI ĐẨY DỮ LIỆU
# ==========================================
print(f"Đang bắt đầu nạp {len(df_final_project)} dòng vào SQL Server...")

try:
    df_final_project.to_sql(
        name=table_name, 
        con=engine, 
        if_exists='replace', 
        index=False, 
        dtype=dtype_dict,
        chunksize=None # fast_executemany sẽ tự quản lý tốt nhất khi để None
    )
    print(f"--- THÀNH CÔNG! Đã lưu vào bảng: {table_name} ---")
except Exception as e:
    print(f"--- THẤT BẠI! Lỗi: {e} ---")

In [9]:
df_final_project = pd.read_sql_query(''' 
SELECT *
  FROM [VTI].[dbo].[Final_Project]
    ; ''', conn)
df_final_project

C:\Users\Tran Minh Huy\AppData\Local\Temp\ipykernel_5968\73694083.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_final_project = pd.read_sql_query('''


,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,...,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
0,4,I am a developer by profession,False,Never,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,22.0,Man,False,Straight / Heterosexual,White or of European descent,False,Appropriate in length,Easy
1,9,I am a developer by profession,True,Once a month or more often,The quality of OSS and closed source software ...,Employed full-time,New Zealand,No,Some college/university study without earning ...,"Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,None,23.0,Man,False,Bisexual,White or of European descent,False,Appropriate in length,Neither easy nor difficult
2,13,I am a developer by profession,True,Less than once a month but more than once per ...,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,United States,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)","Computer science, computer engineering, or sof...",...,Somewhat more welcome now than last year,Tech articles written by other developers;Cour...,28.0,Man,False,Straight / Heterosexual,White or of European descent,True,Appropriate in length,Easy
3,16,I am a developer by profession,True,Never,The quality of OSS and closed source software ...,Employed full-time,United Kingdom,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)",None,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,26.0,Man,False,Straight / Heterosexual,White or of European descent,False,Appropriate in length,Neither easy nor difficult
4,17,I am a developer by profession,True,Less than once a month but more than once per ...,The quality of OSS and closed source software ...,Employed full-time,Australia,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,29.0,Man,False,Straight / Heterosexual,Hispanic or Latino/Latina;Multiracial,False,Appropriate in length,Easy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11547,25134,I am a developer by profession,True,Less than once a month but more than once per ...,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,Ecuador,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Somewhat less welcome now than last year,Tech articles written by other developers,32.0,Man,False,Straight / Heterosexual,Hispanic or Latino/Latina,False,Appropriate in length,Easy
11548,25136,I am a developer by profession,True,Never,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,United States,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers;Cour...,36.0,Man,False,Straight / Heterosexual,White or of European descent,False,Appropriate in length,Difficult
11549,25138,I am a developer by profession,True,Less than once per year,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)","Computer science, computer engineering, or sof...",...,A lot more welcome now than last year,Tech articles written by other developers;Indu...,34.0,Man,False,Straight / Heterosexual,White or of European descent,True,Too long,Easy
11550,25141,I am a developer by profession,True,Less than once a month but more than once per ...,"OSS is, on average, of LOWER quality than prop...",Employed full-time,Switzerland,No,"Secon

3. CLEARNING DATA PROCESS


3.1 Standardization & Handle Missing Value

In [10]:
# 1. Ép kiểu cột Lương (ConvertedComp) về dạng số
df_final_project['ConvertedComp'] = pd.to_numeric(df_final_project['ConvertedComp'], errors='coerce')

# 2. Xử lý thiếu hụt: Dùng Median cho Lương vì lương có độ lệch cực lớn
df_final_project['ConvertedComp'] = df_final_project['ConvertedComp'].fillna(df_final_project['ConvertedComp'].median())

# 3. Xử lý thiếu hụt: Các cột phân loại (Category)
categorical_cols = ['Gender', 'EdLevel', 'Country', 'DevType']
for col in categorical_cols:
    df_final_project[col] = df_final_project[col].fillna('Not specified')

# 4. Xử lý cột Năm kinh nghiệm (YearsCode) - Chuyển chữ thành số
df_final_project['YearsCode'] = df_final_project['YearsCode'].replace({'Less than 1 year': 0, 'More than 50 years': 51})
df_final_project['YearsCode'] = pd.to_numeric(df_final_project['YearsCode'], errors='coerce').fillna(0)

# 5. Xóa trùng lặp
df_final_project.drop_duplicates(inplace=True)

print(f"Kích thước sau khi làm sạch: {df_final_project.shape}")

Kích thước sau khi làm sạch: (11398, 85)


3.2 Check logic (Business Logic) & Outliers

In [12]:
# 1. Lọc tuổi hợp lệ (ví dụ từ 15 đến 90 tuổi)
df_final_project = df_final_project[(df_final_project['Age'] >= 15) & (df_final_project['Age'] <= 90)]

# 2. Năm kinh nghiệm không được lớn hơn (Tuổi - 10)
df_final_project = df_final_project[df_final_project['YearsCode'] <= (df_final_project['Age'] - 10)]

3.3 Detect & Remove Outliers

In [13]:
Q1 = df_final_project['ConvertedComp'].quantile(0.25)
Q3 = df_final_project['ConvertedComp'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_final_project = df_final_project[(df_final_project['ConvertedComp'] >= lower_bound) & (df_final_project['ConvertedComp'] <= upper_bound)]

print(f"Kích thước sau khi làm sạch: {df_final_project.shape}")

Kích thước sau khi làm sạch: (9758, 85)


3.4 Feature Engineering

In [15]:
# Cột LanguageWorkedWith có dạng Java;Python;
# C++. Chúng ta cần biết mỗi dòng có bao nhiêu ngôn ngữ.

# Đếm số lượng ngôn ngữ mỗi người biết
df_final_project['LanguageCount'] = df_final_project['LanguageWorkedWith'].apply(lambda x: len(str(x).split(';')) if pd.notnull(x) else 0)

# Chuẩn hóa tên cột về chữ thường và snake_case (giống sườn bài cũ của bạn)
df_final_project.columns = [col.lower().replace(' ', '_') for col in df_final_project.columns]

4. Save Clean Data to SQL Server

In [20]:
from sqlalchemy import create_engine, urllib

# 1. Tạo chuỗi kết nối cho SQLAlchemy
params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-MV8RPS5;"
    "DATABASE=Data_Topic;" # Đảm bảo đúng tên Database của bạn
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# 2. Xác định kiểu dữ liệu cho các cột đặc biệt (MAX)
# Điều này giúp tránh lại lỗi truncation cũ
from sqlalchemy.types import NVARCHAR

# Tạo dictionary chứa các cột cần để MAX
dtype_dict = {}
long_text_cols = ['devtype', 'languageworkedwith', 'platformworkedwith', 'ethnicity', 'devenviron']

for col in df_final_project.columns:
    if col.lower() in long_text_cols:
        dtype_dict[col] = NVARCHAR(length=None) # length=None tương đương với MAX

# 3. Đẩy dữ liệu lên SQL (Cực kỳ đơn giản và an toàn)
table_name = "final_project_cleaned"

try:
    df_final_project.to_sql(
        name=table_name, 
        con=engine, 
        if_exists='replace', # Tự động xóa bảng cũ và tạo lại
        index=False, 
        dtype=dtype_dict, # Áp dụng NVARCHAR(MAX) cho các cột cần thiết
        chunksize=1000 # Chia nhỏ dữ liệu để đẩy lên nhanh và ổn định
    )
    print(f"--- ĐÃ LƯU THÀNH CÔNG VÀO BẢNG {table_name} ---")
except Exception as e:
    print(f"Lỗi khi lưu: {e}")

ImportError: cannot import name 'urllib' from 'sqlalchemy' (c:\Users\Tran Minh Huy\AppData\Local\Programs\Python\Python313\Lib\site-packages\sqlalchemy\__init__.py)